In [2]:
import os
import shutil

# === CONFIG ===
dataset_a = "Dataset2"  # armed-person-with-rifle
dataset_b = "Dataset3"  # knifeandperson
merged = "Merged_Dataset"

class_map_a = {0: 0, 1: 1, 2: 2}  # Armed-Person, Rifle, Person
class_map_b = {0: 3, 1: 2}        # knife → 3, Person → 2

splits = ["train", "valid", "test"]

# === HELPER FUNCTION ===
def copy_and_remap(src_split_dir, dst_img_dir, dst_lbl_dir, mapping):
    src_img_dir = os.path.join(src_split_dir, "images")
    src_lbl_dir = os.path.join(src_split_dir, "labels")

    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    for img_file in os.listdir(src_img_dir):
        img_src_path = os.path.join(src_img_dir, img_file)
        lbl_src_path = os.path.join(src_lbl_dir, os.path.splitext(img_file)[0] + ".txt")

        # Avoid overwriting duplicate filenames
        base, ext = os.path.splitext(img_file)
        new_img_name = f"{base}_{os.path.basename(src_split_dir)}{ext}"
        new_lbl_name = f"{base}_{os.path.basename(src_split_dir)}.txt"

        img_dst_path = os.path.join(dst_img_dir, new_img_name)
        lbl_dst_path = os.path.join(dst_lbl_dir, new_lbl_name)

        shutil.copy(img_src_path, img_dst_path)

        if os.path.exists(lbl_src_path):
            with open(lbl_src_path, "r") as f:
                lines = f.readlines()
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                old_cls = int(parts[0])
                parts[0] = str(mapping[old_cls])
                new_lines.append(" ".join(parts) + "\n")
            with open(lbl_dst_path, "w") as f:
                f.writelines(new_lines)

# === MERGE PROCESS ===
for split in splits:
    # Dataset A
    copy_and_remap(
        os.path.join(dataset_a, split),
        os.path.join(merged, "images", split),
        os.path.join(merged, "labels", split),
        class_map_a
    )
    # Dataset B
    copy_and_remap(
        os.path.join(dataset_b, split),
        os.path.join(merged, "images", split),
        os.path.join(merged, "labels", split),
        class_map_b
    )

# === NEW data.yaml ===
yaml_content = f"""train: {merged}/images/train
val: {merged}/images/valid
test: {merged}/images/test

nc: 4
names: ['Armed-Person', 'Rifle', 'Person', 'knife']
"""

with open(os.path.join(merged, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("✅ Merge complete! Merged dataset saved to:", merged)

✅ Merge complete! Merged dataset saved to: Merged_Dataset


In [1]:
from ultralytics import YOLO
import os

os.environ["ULTRALYTICS_TENSORBOARD"] = "True"


# Load the base YOLOv8 model
model = YOLO("runs/detect/train3/weights/last.pt")  # or yolov8s.pt, etc.

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
model.train(
    data="Merged_Dataset/data.yaml",
    epochs=100,
    imgsz=640,
    batch=8,
    device=0,
    cache=True,
    amp=True,
    save_period=5,
    val=True
)

New https://pypi.org/project/ultralytics/8.3.179 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Merged_Dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/train3/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nb

100%|██████████| 755k/755k [00:00<00:00, 3.51MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

Model summary: 129 layers, 3,011,628 parameters, 3,011,612 gradients, 8.2 GFLOPs

Transferred 355/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 1.8±0.2 ms, read: 21.5±2.8 MB/s, size: 62.0 KB)


train: Scanning /app/app/Model/Merged_Dataset/labels/train.cache... 10320 images, 38 backgrounds, 0 corrupt: 100%|██████████| 10320/10320 [00:00<?, ?it/s]

...

                   all        327       1042      0.648      0.488      0.528      0.294



100 epochs completed in 3.305 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.3MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:03<00:00,  6.41it/s]


                   all        327       1042      0.645      0.496      0.528      0.293
          Armed-Person        260        376      0.852      0.522       0.72      0.436
                 Rifle        274        405      0.855      0.306      0.484       0.22
                Person        143        257      0.575      0.405      0.468      0.282
                 knife          4          4      0.299       0.75      0.439      0.234
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x791a98525ed0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [13]:
import pandas as pd
import matplotlib.pyplot as plt

# Load training results
df = pd.read_csv('runs/detect/train3/results.csv')

# Show first rows
print(df.head())
print(df.columns)


   epoch     time  train/box_loss  train/cls_loss  train/dfl_loss  \
0      1  129.458         1.23173         2.05414         1.45645   
1      2  250.698         1.19663         1.75207         1.42017   
2      3  367.712         1.17812         1.67987         1.41249   
3      4  490.113         1.13795         1.59827         1.38363   
4      5  603.206         1.10301         1.53592         1.36118   

   metrics/precision(B)  metrics/recall(B)  metrics/mAP50(B)  \
0               0.33980            0.22923           0.20926   
1               0.48883            0.21957           0.17545   
2               0.56840            0.20941           0.19406   
3               0.56172            0.23996           0.22185   
4               0.32547            0.26360           0.23680   

   metrics/mAP50-95(B)  val/box_loss  val/cls_loss  val/dfl_loss    lr/pg0  \
0              0.09463       1.94200       2.56550       2.16813  0.000416   
1              0.07354       2.04179       2